Aclaracion: para no hacer el archivo largo y repetitivo se fueorn pisando las cosas anteriores a medida que se encontraba algun hiperparametro. El orden en el que se fue buscando cada hiperparametro se encuentra en el informe

In [ ]:
! pip3 install bayesian_optimization
! pip3 install xgboost
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from bayes_opt import BayesianOptimization
from bayes_opt import SequentialDomainReductionTransformer
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
train_values = pd.read_csv('train_values_short1.csv', index_col='building_id')
train_labels = pd.read_csv('train_labels.csv', index_col='building_id')

In [177]:
def target(resultado):
    return resultado['target']

def mostrar(resultados, min=0):
    
    a_borrar = []
    for resultado in resultados:
        if(resultado['target']<min):
            a_borrar.append(resultado)
    resultados_importantes = [a for a in resultados if a not in a_borrar]
    
    resultados_importantes.sort(reverse=True, key=target)
    
    for resultado in resultados_importantes:
        print("target: " , resultado['target'])
        for key,value in resultado['params'].items():
            print("\t\t", key, ':', value)
        print()

In [178]:
def funcion_a_optimizar(subsample, colsample_bytree):
    x = train_values
    le = LabelEncoder()
    y = le.fit_transform(y=train_labels.values.ravel())
    xgb = XGBClassifier( objective = "multi:softmax", use_label_encoder=False, seed=30,
                         max_depth=9,
                         min_child_weight=6,
                         gamma=0.2554138025988315,
                         subsample=subsample,
                         colsample_bytree=colsample_bytree,
                         learning_rate =0.1, 
                         n_estimators=240, 
                         )
                       
    return cross_val_score(xgb, x, y, scoring='f1_micro', cv=10).mean()

In [179]:
limites = {"subsample":(0.5,1), "colsample_bytree":(0.5,1) }

In [182]:
optimizer = BayesianOptimization(
    f=funcion_a_optimizar,
    random_state=130,
    verbose=5,
    pbounds=limites,
    bounds_transformer=SequentialDomainReductionTransformer(),
)

#load_logs(optimizer, "./anteriores.json")
logger = JSONLogger(path="./con_feature_engiering_colsample_y_subsample.json")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [183]:
optimizer.maximize(
    init_points=5,
    n_iter=40,
)

[20:25:59] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:26:29] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [ ]:
optimizer.probe({"subsample":1, "colsample_bytree":1 }, lazy=True)

In [ ]:
optimizer.probe({"subsample":0.8, "colsample_bytree":0.8 }, lazy=True)

In [184]:
mostrar(optimizer.res)

target:  0.7585197330134774
		 max_depth : 8.399244502867223
		 min_child_weight : 1.1037597467970075

